# 构建不同任务的数据集

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-05-25  

# Task1. 预测是酶还是非酶数据集构建



## 任务简介
该任务通过给定蛋白序列，预测该该蛋白是酶还是非酶。本任务所使用的数据集为Sport，对数据集的数据中进行学习，然后对新给定的蛋白序列数据预测是酶还是非酶。


## 数据统计
- 数据源Sprot，共有数据564,638条，其中有EC号的数据270,236条，无EC号的数据294402条, 具体数据如下：

| 项目                           | 数据    |
| -------------------------------- | ---------:|
|  原始Sprot                     |  564,638  |
|  去重复后的Sprot            |  476,664  |
|  >50bp                           |  467,097  |
|  无EC号                        |  239,019  |
|  有3级以上EC号的           |  219,227  |
|  有EC号但不满足三级      |  8,851    |
|  (无EC号 + 有EC号但不满足三级 =(239,019 + 8,851 = 247,870)比对有EC号的返回数据条数  |  32,238    |
|  显著性大于40%的数据条数  |  13,885   |
|  筛选后的数据总数          |  453,212   |
|  筛选后的酶数据           |  219,227  |
|  筛选后的非酶数据        |  233,985  |


- 将数据集中的所有数据按照时间排序，～90%作为训练集，～10%作为测试集，找到对应时间节点为2009年12月14日。
- 以2009年12月14日为时间节点，之前的数据为训练集，之后的数据为测试集，具体数据集统计如下： 





|     Items    | 酶       |   非酶    |合计                               |
| ------------ | --------| --------- |----------------------------------|
| 训练集        | 198,692 | 208,391   | 407,083（407,083/453,212=89.82%)  |
| 测试集        | 20,535  | 25,594    | 46,129 (46,129/453,212=10.18% )  |







## 数据集构建方法

* 根据蛋白注释信息，将Sprot中的蛋白分为「酶」和 「非酶」。
* 为了保证蛋白的唯一性，按照序列对Sprot中的数据进行去重除了，以保证数据集中序列的唯一性。
* 将注释中含有3级及三级以上EC号的蛋白做为酶数据，收录于数据集中，当中正样本 。
* 将注释中不含有EC号的数据作为非酶数据，并进行如下处理后收录于数据集中，当做是负样本。
> 1. 非酶数据与酶数据进行同源比对(diamoind)，过滤掉相似性>40%的数据
> 2. 过滤掉<50bp的核算片段
> 3. 对酶数据进行CD——HIT比较，按照40%的阈值进行去冗余<span style='background:yellow'>（？有必要吗）</span>

## 1. 导入必要的包

In [2]:
import numpy as np
import pandas as pd
import random
import os
import time
import datetime
import sys
from tqdm import tqdm

from functools import reduce
import matplotlib.pyplot as plt

sys.path.append("./tools/")
import funclib

from pyecharts.globals import CurrentConfig, OnlineHostType
CurrentConfig.ONLINE_HOST = OnlineHostType.NOTEBOOK_HOST
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.faker import Faker
from pyecharts.globals import ThemeType


%load_ext autoreload
%autoreload 2

## 2. 读入数据

In [6]:
#加载数据并转换时间格式
sprot = pd.read_csv('./data/sprot_full.tsv', sep='\t',names=funclib.table_head) #读入文件
sprot.date_integraged = pd.to_datetime(sprot['date_integraged'])
sprot.date_sequence_update = pd.to_datetime(sprot['date_sequence_update'])
sprot.date_annotation_update = pd.to_datetime(sprot['date_annotation_update'])
sprot = sprot.sort_values(by='date_integraged', ascending=True)
sprot.head(2)

,id,name,isemzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,P02802,MT1_MOUSE,False,False,1,-,0,1986-07-21,1986-07-21,2021-04-07,MDPNCSCSTGGSCTCTSSCACKNCKCTSCKKSCCSCCPVGCSKCAQ...,61
2982,P61852,SODC_DROSI,True,False,1,1.15.1.1,4,1986-07-21,2007-01-23,2020-12-02,MVVKAVCVINGDAKGTVFFEQESSGTPVKVSGEVCGLAKGLHGFHV...,153


## 3. 数据去重, 并筛选大于50bp的序列

In [7]:
# 统计重复数据
table_repeat=pd.value_counts(sprot.seq, sort=True)
table_repeat = pd.DataFrame(table_repeat)
table_repeat['sequence'] = table_repeat.index
table_repeat.columns=['repeat', 'seq']
table_repeat = table_repeat.reset_index(drop=True)

#准备画图数据
figure_data = pd.value_counts(table_repeat.repeat, sort=True)
figure_data = pd.DataFrame(figure_data)
figure_data['x'] = figure_data.index
figure_data.columns=['y', 'x']

bar = (
    Bar(init_opts=opts.InitOpts(width="1700px",
                                height="750px",
                                page_title="sprot",
                                theme=ThemeType.CHALK))
    .add_xaxis(list(figure_data.x))
    .add_yaxis("重复次数", list(figure_data.y))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="SPROT重复序列数据统计"),
        datazoom_opts=opts.DataZoomOpts()
    )
)
bar.load_javascript()
bar.render_notebook()


In [8]:
# 去除重复数据, 选择 >50bp的序列
sprot = sprot.drop_duplicates(subset=['seq'], keep='first', inplace=False)
sprot = sprot.reset_index(drop=True)
base_data = sprot[sprot.seqlength >= 50]
base_data.head(2)

,id,name,isemzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,P02802,MT1_MOUSE,False,False,1,-,0,1986-07-21,1986-07-21,2021-04-07,MDPNCSCSTGGSCTCTSSCACKNCKCTSCKKSCCSCCPVGCSKCAQ...,61
1,P61852,SODC_DROSI,True,False,1,1.15.1.1,4,1986-07-21,2007-01-23,2020-12-02,MVVKAVCVINGDAKGTVFFEQESSGTPVKVSGEVCGLAKGLHGFHV...,153


## 4. 序列比对,对非酶数据选取同源率 <40% 的序列

In [9]:
emzyme_data = base_data[base_data.isemzyme]
noemzyme_data = base_data[base_data.isemzyme == False]
res_data = funclib.getblast(emzyme_data, noemzyme_data)

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1


In [10]:
# 按阈值筛选
notinlist = list(res_data[res_data.pident>40].id)

# 过滤同源性高的序列
base_data = base_data[~base_data.id.isin(notinlist)]
base_data = base_data.reset_index(drop=True)

## 5. 保存文件

In [ ]:
h5 = pd.HDFStore('./data/emzyme_noemzyme.h5', 'w', complevel=4, complib='blosc')
h5['data'] = base_data
h5.close()

In [13]:
# 原始数据

uni_data = np.load('./tasks/task1/data/emzyme_noemzyme_unirep.npy', allow_pickle=True)
uni_data =pd.DataFrame([i for k in uni_data for i in k ])
uni_data[7] = pd.to_datetime(uni_data[7])

# 补充数据
uni_data_supply = np.load('./tasks/task1/data/emzyme_noemzyme_unirep_supply.npy', allow_pickle=True)
uni_data_supply =pd.DataFrame([i for k in uni_data_supply for i in k ])
uni_data_supply[7] = pd.to_datetime(uni_data_supply[7])

# 拼合数据
uni_res = uni_data.append(uni_data_supply,ignore_index=True)
ids = list(base_data.id)
uni_res = uni_res[uni_res[0].isin(ids)]
uni_res = uni_res.sort_values(by=7)
uni_res = uni_res.reset_index(drop=True)

# 保存全部数据
h5 = pd.HDFStore('./data/emzyme_noemzyme_with_unirep.h5', 'w', complevel=4, complib='blosc')
h5['data'] = uni_res
h5.close()


# 保存前部数据
h5 = pd.HDFStore('./data/emzyme_noemzyme_with_unirep.h5', 'w', complevel=4, complib='blosc')
h5['data'] = uni_res.iloc[:,0:12]
h5.close()

,0,1,2,3,4,5,6,7,8,9,...,1902,1903,1904,1905,1906,1907,1908,1909,1910,1911
0,P02802,MT1_MOUSE,False,False,1,-,0,1986-07-21,1986-07-21,2021-04-07,...,-0.002212,0.119389,-0.019647,-0.016905,-0.532317,0.008659,-0.011296,0.001098,0.794472,0.210124
1,P61852,SODC_DROSI,True,False,1,1.15.1.1,4,1986-07-21,2007-01-23,2020-12-02,...,0.189573,-0.039781,-0.457674,0.018009,-0.097954,-0.030856,0.060694,0.015370,0.012495,0.000180
2,P00925,ENO2_YEAST,True,False,1,4.2.1.11,4,1986-07-21,2007-01-23,2021-04-07,...,-0.009811,0.000229,-0.227972,0.185412,-0.106373,0.060093,0.007852,0.061177,-0.071324,-0.030171


In [10]:
emzyme_noemzyme_data=pd.read_hdf('./data/emzyme_noemzyme_with_unirep.h5',key='data')
emzyme_noemzyme_head = funclib.table_head + ['f'+str(i) for i in range(1, 1901) ]
emzyme_noemzyme_data.columns = emzyme_noemzyme_head

In [12]:
t_thres = datetime.datetime(2009, 12, 14, 0, 0)

#训练集
train = emzyme_noemzyme_data[emzyme_noemzyme_data.date_integraged <= t_thres ].sort_values(by='date_integraged')
#测试集
test = emzyme_noemzyme_data[emzyme_noemzyme_data.date_integraged > t_thres ].sort_values(by='date_integraged')

# 训练集
h5 = pd.HDFStore('./tasks/task1/data/train.h5', 'w', complevel=4, complib='blosc')
h5['data'] = train
h5.close()

#测试集
h5 = pd.HDFStore('./tasks/task1/data/test.h5', 'w', complevel=4, complib='blosc')
h5['data'] = test
h5.close()
# train.iloc[:,np.r_[0, 2, 7,10:12, 12:1912]]

# Task2. 预测是单功能酶还是多功能数据集构建



## 任务简介
该任务通过给定酶序列，预测该酶还是多功能酶还是单功能酶。本任务所使用的数据集为Sport，对数据集的数据中进行学习，然后对新给定的酶进行单功能/多功能预测。


## 数据统计
- 遵照任务1找到的时间节点对数据进行划分
- 以2009年12月14日为时间节点，之前的数据为训练集，之后的数据为测试集，具体数据集统计如下： 


|     Items    | 单功能酶       |   多功能非酶    |合计                               |
| ------------ | --------| --------- |----------------------------------|
| 训练集        | 185,453 | 13,239   | 198,692（198,692/219,227=90.63%)  |
| 测试集        | 18,868  | 25,594    | 20,535 (20,535/219,227=9.37% )  |


## 数据集构建方法

* 根据蛋白注释信息与之前划定的酶与非酶数据集，将「酶」数据进行分类。
* 有1个EC号的被定义为「单功能酶」，有多个EC号的被定义为「多功能酶」。


In [16]:
emzyme_data = emzyme_noemzyme_data[emzyme_noemzyme_data.isemzyme]

In [19]:
#训练集
train = emzyme_data[emzyme_data.date_integraged <= t_thres ].sort_values(by='date_integraged')
#测试集
test = emzyme_data[emzyme_data.date_integraged > t_thres ].sort_values(by='date_integraged')

# 训练集
h5 = pd.HDFStore('./tasks/task2/data/train.h5', 'w', complevel=4, complib='blosc')
h5['data'] = train
h5.close()

#测试集
h5 = pd.HDFStore('./tasks/task2/data/test.h5', 'w', complevel=4, complib='blosc')
h5['data'] = test
h5.close()

In [28]:
20535/219227

0.09367003151984017

In [26]:
emzyme_data.shape

(219227, 1912)